# Импорты

In [1]:
# !pip install psycopg2 -q

In [2]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
import sqlalchemy
import time
import socket
import sys

from creds import db_config

# Подключение к базе и получение данных

In [3]:
# Импортируем конфигурацию из отдельного файла
try:
    display("Конфигурация загружена из creds.py")
except ImportError:
    display("Файл creds.py не найден")
    db_config = {
        'user': input("Введите имя пользователя: "),
        'pwd': input("Введите пароль: "),
        'host': input("Введите хост: "),
        'port': input("Введите порт: "),
        'db': input("Введите название БД: ")
    }

'Конфигурация загружена из creds.py'

In [4]:
def check_host_availability():
    """Проверка доступности хоста"""
    try:
        socket.create_connection((db_config['host'], db_config['port']), timeout=10)
        display("Хост доступен")
        return True
    except socket.error as e:
        display(f"Хост недоступен: {e}")
        sys.exit("ОСТАНОВКА: Не удалось подключиться к базе данных. Дальнейшее выполнение невозможно.")

check_host_availability()

'Хост недоступен: timed out'

SystemExit: ОСТАНОВКА: Не удалось подключиться к базе данных. Дальнейшее выполнение невозможно.

e:\my_github\yandex_practicum_ds\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
connection_string = f"postgresql://{db_config['user']}:{db_config['pwd']}@{db_config['host']}:{db_config['port']}/{db_config['db']}"
engine = create_engine(connection_string)

def load_all_tables(engine):
    """Загружает все таблицы из базы данных"""
    
    tables = {
        'collisions': 'SELECT * FROM collisions',
        'parties': 'SELECT * FROM parties', 
        'vehicles': 'SELECT * FROM vehicles'
    }
    
    dataframes = {}
    
    for table_name, query in tables.items():
        try:
            display(f"Загружаем таблицу {table_name}...")
            dataframes[table_name] = pd.read_sql(query, engine)
            display(f"{table_name}: {dataframes[table_name].shape[0]} строк, {dataframes[table_name].shape[1]} столбцов")
        except Exception as e:
            display(f"Ошибка при загрузке {table_name}: {e}")
    
    return dataframes

display("Начинаем загрузку данных из базы...")
df_dict = load_all_tables(engine)

# Сохранение в отдельные переменные для удобства
collisions = df_dict['collisions']
parties = df_dict['parties']
vehicles = df_dict['vehicles']

display("\n" + "="*50)
display("СВОДКА ПО ДАННЫМ:")
display("="*50)
display(f"collisions: {collisions.shape}")
display(f"parties:    {parties.shape}")
display(f"vehicles:   {vehicles.shape}")

display("\nПервые 3 строки каждой таблицы:")
display("\ncollisions:")
display(collisions.head(3))

print("\nparties:")
display(parties.head(3))

display("\nvehicles:")
display(vehicles.head(3))

display("\nИнформация о таблицах:")
display("\ncollisions info:")
collisions.info()

display("\nparties info:")
parties.info()

display("\nvehicles info:")
vehicles.info()